In [1]:
!pip install torch
!pip install sentence-transformers
!pip install transformers
!pip install numpy
!pip install pandas
!pip install scikit-learn

In [2]:
import os
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load Data
def load_lyrics_dataset(file_path):
    df = pd.read_csv(file_path)
    return df

def retrieve_top_k_songs(query, song_embeddings, song_metadata, bi_encoder, k=10):
    """Retrieve top-k songs based on cosine similarity."""
    query_embedding = bi_encoder.encode_texts([query])
    similarities = util.pytorch_cos_sim(query_embedding, song_embeddings)
    top_k_indices = torch.topk(similarities, k=k).indices[0]
    return [song_metadata[i] for i in top_k_indices.tolist()]

In [4]:
class BiEncoder:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.model = SentenceTransformer(model_name)

    def encode_texts(self, texts):
        """Used for encoding lyrics into embeddings."""
        return self.model.encode(texts, convert_to_tensor=True, show_progress_bar=True)

class CrossEncoder:
    def __init__(self, model_name='cross-encoder/ms-marco-MiniLM-L-6-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def rank_candidates(self, query, candidates):
        inputs = [
            self.tokenizer(query, candidate, return_tensors='pt', truncation=True, max_length=512, padding=True)
            for candidate in candidates
        ]
        scores = []
        for input_pair in inputs:
            with torch.no_grad():
                logits = self.model(**input_pair).logits
            scores.append(logits.item())
        ranked_indices = np.argsort(scores)[::-1]
        return ranked_indices

In [9]:
# Main Pipeline
def song_retrieval_pipeline(query, dataset_path, bi_encoder_model='sentence-transformers/all-mpnet-base-v2',
                            cross_encoder_model='cross-encoder/ms-marco-MiniLM-L-6-v2', k=10):
    data = load_lyrics_dataset(dataset_path)
    lyrics = data['lyrics'].tolist()
    song_metadata = data[['track_name', 'artist_name', 'lyrics']].to_dict(orient='records')

    # Bi-Encoder
    bi_encoder = BiEncoder(model_name=bi_encoder_model)
    song_embeddings = bi_encoder.encode_texts(lyrics)

    # Initial retrieval
    top_k_songs = retrieve_top_k_songs(query, song_embeddings, song_metadata, bi_encoder, k)
    top_k_lyrics = [song['lyrics'] for song in top_k_songs]

    # Cross-Encoder
    cross_encoder = CrossEncoder(model_name=cross_encoder_model)
    ranked_indices = cross_encoder.rank_candidates(query, top_k_lyrics)

    # Re-ranking
    re_ranked_songs = [top_k_songs[i] for i in ranked_indices]
    return re_ranked_songs


In [12]:
# Main program
dataset_path = "dataset.csv"
query = "want a little bit heart"

if os.path.exists(dataset_path):
    print("Dataset obtained.")
    results = song_retrieval_pipeline(query, dataset_path)
    print("Top retrieved songs:")
    for idx, song in enumerate(results):
        print(f"{idx + 1}. {song['track_name']} by {song['artist_name']}")
else:
    print("Dataset path does not exist.")

Dataset obtained.


Batches:   0%|          | 0/887 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top retrieved songs:
1. your hearts not in it by janie fricke
2. hello my old heart by the oh hellos
3. all of me (loves all of you) by george strait
4. promises, promises by jerry vale
5. two faces have i by lou christie
6. from me to you by the beatles
7. from me to you by del shannon
8. heartfull of soul by the yardbirds
9. somebody by depeche mode
10. love by john lennon
